In this notebook, I will build a simple chatbot using LangGraph that can initiate conversations, integrate external tools, and retain conversation history.

Install dependencies

In [1]:
!pip3 install langgraph typing_extensions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17/17 [langgraph]17 [langgraph-checkpoint]


To get started with building the chatbot, the first thing we need to do is initialize a state graph. A state graph is a core concept in LangGraph that models the flow of a conversation as a state machine — a system where different 'states' represent different stages in a process.

In our case, each state is defined by a node. A node can be anything from a simple Python function, a call to a language model (LLM), or an integration with an external tool (like a calculator or a web search API). These nodes define the individual operations or logic that the chatbot performs.

Once we define the nodes, we connect them using edges, which specify how the conversation should transition from one node to another based on the current input or output. This structure allows us to build flexible, dynamic workflows where the chatbot can respond intelligently, call tools when needed, and maintain memory across interactions.

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages

Creating a graph using stategraph

In [3]:
class State(TypedDict):
    """State for the graph."""
    messages: Annotated[list, add_messages]
graph_builder = StateGraph(State)


Install langchain aws so that we can connect all the models hosted in AWS bedrock

In [4]:
pip install -U "langchain[aws]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 17.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 15.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [langchain]/9 [langchain]
Note: you may need to restart the kernel to use updated packages.


The previous step initialized a new graph builder. Now, we can begin creating nodes and connecting them with edges, allowing the chatbot to transition between different stages or functions.

To define a new node, we create a Python function that accepts the current state as input. This state includes the latest data from the conversation. The function performs its specific task—such as generating a response or calling a tool—and then returns an updated state.

Within the function, you'll write the logic needed to handle that part of the conversation. Once the task is completed, the state is modified to include any new messages or data, which can then be used by the next node in the graph.

In [5]:
from langchain.chat_models import init_chat_model
model = init_chat_model("anthropic.claude-3-5-sonnet-20240620-v1:0", model_provider="bedrock_converse")
def chatbot(state: State) -> State:
    """Chatbot function to handle messages."""
    # response = model.invoke(state["messages"])
    # print(response)
    # state["messages"].append(response)
    # return state
    return {"messages": [model.invoke(state["messages"])]}
graph_builder.add_node("chatbot", chatbot)


ValidationError: 1 validation error for ChatBedrockConverse
  Value error, Could not load credentials to authenticate with AWS client. Please check that the specified profile name and/or its credentials are valid. Service error: You must specify a region. [type=value_error, input_value={'model': 'anthropic.clau...sable_streaming': False}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error